In [3]:
import numpy as np
import matplotlib.pyplot as plt

#This function imports the data in the NACAxxxx file and returns the alpha, CL and CM values
def import_data(filename):
    PATH = r'C:/Users/mtaha/OneDrive/Belgeler/GitHub/FoilSolver/Surrogate_Model/XFoil_data/NACAxxxx'
    filename = PATH + '\\' + filename + '.txt'
    data = np.genfromtxt(filename, skip_header=12)
    alpha = data[:,0]
    CL = data[:,1]
    CM = data[:,4]
    return alpha, CL, CM

x_matrix = np.zeros((9999*15, 4))
CL_matrix = np.zeros((9999*15, 1))
CM_matrix = np.zeros((9999*15, 1))

for i in range(9999):
    if np.mod(i,100) != 0:
        NAME = str(i).zfill(4)
        first_param = int(NAME[0])
        second_param = int(NAME[1])
        third_param = int(NAME[2:])
        alpha, CL, CM = import_data('NACA' + NAME)
        x_matrix[i*15:(i+1)*15,0] = first_param
        x_matrix[i*15:(i+1)*15,1] = second_param
        x_matrix[i*15:(i+1)*15,2] = third_param
        x_matrix[i*15:(i+1)*15,3] = alpha
        CL_matrix[i*15:(i+1)*15,0] = CL
        CM_matrix[i*15:(i+1)*15,0] = CM


In [ ]:

#This function trains the data with x_matrix and CL_matrix with feedforward neural network

def train_data(x_matrix, CL_matrix):
    from sklearn.neural_network import MLPRegressor
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score
    x_train, x_test, y_train, y_test = train_test_split(x_matrix, CL_matrix, test_size=0.2, random_state=42)
    mlp = MLPRegressor(hidden_layer_sizes=(100,100,100), max_iter=1000)
    mlp.fit(x_train, y_train)
    predictions = mlp.predict(x_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    accuracy = mlp.score(x_test, y_test)
    print('MSE: ', mse)
    print('R2: ', r2)
    print('Accuracy: ', accuracy)
    return mlp

def predict_CL(mlp, first_param, second_param, third_param, alpha):
    x = np.array([first_param, second_param, third_param, alpha])
    x = x.reshape(1, -1)
    prediction = mlp.predict(x)
    return prediction

model = train_data(x_matrix, CL_matrix)
pred = predict_CL(model, 0, 0, 12, 5)
print(pred)


In [62]:
#This function trains the data with x_matrix and CM_matrix by using polynomial regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


def train_data_poly(x_matrix, CL_matrix):

    x_train, x_test, y_train, y_test = train_test_split(x_matrix, CL_matrix, test_size=0.2, random_state=42)
    poly = PolynomialFeatures(degree=9)
    x_poly = poly.fit_transform(x_train)
    poly.fit(x_poly, y_train)
    lin2 = LinearRegression()
    lin2.fit(x_poly, y_train)
    predictions = lin2.predict(poly.fit_transform(x_test))
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    accuracy = lin2.score(poly.fit_transform(x_test), y_test)
    print('MSE: ', mse)
    print('R2: ', r2)
    print('Accuracy: ', accuracy)
    return lin2

def predict_CL(lin2, first_param, second_param, third_param, alpha):
    x = np.array([first_param, second_param, third_param, alpha])
    x = x.reshape(1, -1)
    poly = PolynomialFeatures(degree=9)
    x_poly = poly.fit_transform(x)
    prediction = lin2.predict(x_poly)
    return prediction


In [63]:

model2 = train_data_poly(x_matrix, CL_matrix)

pred2 = predict_CL(model2, 0, 0, 12, 9)
print(pred2)

MSE:  0.005045273223678188
R2:  0.998405503762391
Accuracy:  0.998405503762391
[[0.73404049]]
